# Pandas

The `numpy` module is excellent for numerical computations, but to handle missing data or arrays with mixed types takes more work. The `pandas` module is currently the most widely used tool for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular `pandas` features:

* A fast and efficient "DataFrame" object for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Intelligent data alignment and integrated handling of missing data;
* Intelligent label-based slicing, fancy indexing, and subsetting of large data sets;
* Aggregating or transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Time series-functionalities;
* Highly optimized for performance, with critical code paths written in Cython or C.


In [2]:
import pandas as pd
import numpy as np

## Series

Series are completely equivalent to 1D array but with axis labels and the possibility to store heterogeneous elements. Of paramount importance are the time-series, used to define time evolutions of a phenomenon. 


In [2]:
from string import ascii_lowercase as letters

# Creating a series, accessing indexes, values and values by their index 
xs = pd.Series(np.arange(10)*0.5, index=tuple(letters[:10]))
print ("xs:", xs,'\n')
print ("xs indexes:",xs.index,'\n')
# Values of the Series are actually a numpy array
print ("xs values:", xs.values, type(xs.values),'\n')
print (xs['f'], xs.f, xs.h, '\n')
print (xs[['d', 'f', 'h']], '\n')
print (type(xs[['d', 'f', 'h']]), '\n')

xs: a    0.0
b    0.5
c    1.0
d    1.5
e    2.0
f    2.5
g    3.0
h    3.5
i    4.0
j    4.5
dtype: float64 

xs indexes: Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype='object') 

xs values: [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5] <class 'numpy.ndarray'> 

2.5 2.5 3.5 

d    1.5
f    2.5
h    3.5
dtype: float64 

<class 'pandas.core.series.Series'> 



In [3]:
# Extracting elements and operations: same as numpy array
print (xs[:3],'\n')
print (xs[7:], '\n')
print (xs[::3], '\n')
print (xs[xs>3], '\n')
print (np.exp(xs), '\n')
print (np.mean(xs), np.std(xs), '\n')

a    0.0
b    0.5
c    1.0
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a    0.0
d    1.5
g    3.0
j    4.5
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a     1.000000
b     1.648721
c     2.718282
d     4.481689
e     7.389056
f    12.182494
g    20.085537
h    33.115452
i    54.598150
j    90.017131
dtype: float64 

2.25 1.4361406616345072 



In [4]:
# Series can be created from python dictionary too.
# Not that the elements can be whatever!
d = {'b' : 1, 'a' : 'cat', 'c' : [2,3]}
pd.Series(d)



b         1
a       cat
c    [2, 3]
dtype: object

A key difference between Series and ndarray is that operations between Series automatically align the data based on label. Thus, you can write computations without considering whether the Series involved have the same labels.

In [5]:
s = pd.Series(np.random.randn(5), index=tuple(letters[:5]))
print(s)
s = s[1:] + s[:-1]
print(s)

a   -0.129715
b    0.913202
c   -1.528948
d    1.976067
e    1.729426
dtype: float64
a         NaN
b    1.826403
c   -3.057897
d    3.952135
e         NaN
dtype: float64


### Time series

Time series are very often used to profile the behaviour of a quantity as a function of time. Pandas as a special index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`

In [6]:
# to define a date, the datetime module is very useful
import datetime as dt
date = dt.date.today()
print(date)

date = dt.datetime(2020,11,9,14,45,10,15)
print (date)

# otherwise, several notations are interpreted too
date = 'Nov 9 2020'
# or alternatively
date = '9/11/2020 14:45:00'
print (date)

days = pd.date_range(date, periods=7, freq='D')
print (days)

seconds = pd.date_range(date, periods=3600, freq='s')
print (seconds)


2020-11-09
2020-11-09 14:45:10.000015
9/11/2020 14:45:00
DatetimeIndex(['2020-09-11 14:45:00', '2020-09-12 14:45:00',
               '2020-09-13 14:45:00', '2020-09-14 14:45:00',
               '2020-09-15 14:45:00', '2020-09-16 14:45:00',
               '2020-09-17 14:45:00'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2020-09-11 14:45:00', '2020-09-11 14:45:01',
               '2020-09-11 14:45:02', '2020-09-11 14:45:03',
               '2020-09-11 14:45:04', '2020-09-11 14:45:05',
               '2020-09-11 14:45:06', '2020-09-11 14:45:07',
               '2020-09-11 14:45:08', '2020-09-11 14:45:09',
               ...
               '2020-09-11 15:44:50', '2020-09-11 15:44:51',
               '2020-09-11 15:44:52', '2020-09-11 15:44:53',
               '2020-09-11 15:44:54', '2020-09-11 15:44:55',
               '2020-09-11 15:44:56', '2020-09-11 15:44:57',
               '2020-09-11 15:44:58', '2020-09-11 15:44:59'],
              dtype='datetime64[ns]', lengt

To learn more about the frequency strings, please see this [link](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)


Timestamped data is the most basic type of time series data that associates values with points in time. For pandas objects it means using the points in time.

functions like `pd.to_datetime` can be used, for instance, when reading information as string from a dataset

In [7]:
tstamp = pd.Timestamp(dt.datetime(2020, 11, 9))

# internally it counts the nanoseconds from January 1st 19
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print(tstamp.value)

# when creating a timestamp the format can be explicitly passed
ts = pd.to_datetime('2010/11/12', format='%Y/%m/%d')
print (type(ts))
print (ts)
ts = pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M')
print (ts)
print (ts.value)



1604880000000000000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2010-11-12 00:00:00
2010-11-12 00:00:00
1289520000000000000


A standard series can be created and (range of) elements can be used as indexes

In [8]:
tseries = pd.Series(np.random.normal(10, 1, len(days)), index=days)
# Extracting elements
print (tseries[0:4], '\n')
print (tseries['2020-11-9':'2020-11-11'], '\n') # Note - includes end time


2020-09-11 14:45:00     9.809542
2020-09-12 14:45:00    10.932428
2020-09-13 14:45:00    10.259516
2020-09-14 14:45:00    11.066978
Freq: D, dtype: float64 

Series([], Freq: D, dtype: float64) 



`pd.to_datetime` can also be used to create a `DatetimeIndex`:

In [9]:
pd.to_datetime([1, 2, 3, 4], unit='D', origin=pd.Timestamp('1980-02-03'))

DatetimeIndex(['1980-02-04', '1980-02-05', '1980-02-06', '1980-02-07'], dtype='datetime64[ns]', freq=None)

## DataFrame

A pandas DataFrame is like a simple tabular spreadsheet. For future reference (or for people already familiar with R), a pandas DataFrame is very similar to the R DataFrame.

Each column in a DataFrame is a Series object.

The element can be whatever, missing data are dealt with too (as NaN)

### DataFrame creation

A DataFrame can be created implicitly, with, e.g., a DatatimeIndex object as index:

In [10]:
entries=10
dates=pd.date_range('11/9/2020 14:45:00',freq='h', periods=entries)
df = pd.DataFrame(np.random.randn(entries,4), index=dates, columns=['A','B','C','D'])
df


,A,B,C,D
2020-11-09 14:45:00,-0.973261,1.000826,0.394591,0.122288
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,-0.039428
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,-0.808769
2020-11-09 17:45:00,0.731740,1.072999,0.702288,-0.919768
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,0.324980
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,-0.610307
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,-0.186220
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,1.093060
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,-0.283645
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,-0.073944


or by means of a dictionary:


In [11]:
df2 = pd.DataFrame(
    { 'A' : 1.,
      'B' : pd.Timestamp('20130102'),
      'C' : pd.Series(1,index=range(4),dtype='float32'),
      'D' : np.arange(7,11),
      'E' : pd.Categorical(["test","train","test","train"]),
    }
    )
df2

,A,B,C,D,E
0,1.0,2013-01-02,1.0,7,test
1,1.0,2013-01-02,1.0,8,train
2,1.0,2013-01-02,1.0,9,test
3,1.0,2013-01-02,1.0,10,train


### Viewing Data

In [12]:
df.head()

,A,B,C,D
2020-11-09 14:45:00,-0.973261,1.000826,0.394591,0.122288
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,-0.039428
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,-0.808769
2020-11-09 17:45:00,0.731740,1.072999,0.702288,-0.919768
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,0.324980


In [13]:
df.tail(4)

,A,B,C,D
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,-0.186220
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,1.093060
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,-0.283645
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,-0.073944


In [14]:
df.index

DatetimeIndex(['2020-11-09 14:45:00', '2020-11-09 15:45:00',
               '2020-11-09 16:45:00', '2020-11-09 17:45:00',
               '2020-11-09 18:45:00', '2020-11-09 19:45:00',
               '2020-11-09 20:45:00', '2020-11-09 21:45:00',
               '2020-11-09 22:45:00', '2020-11-09 23:45:00'],
              dtype='datetime64[ns]', freq='H')

In [15]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [16]:
df.values

array([[-0.97326114,  1.00082617,  0.39459063,  0.12228755],
       [ 0.40507126, -0.95481516,  0.29993406, -0.03942793],
       [ 0.34500332, -1.2519749 , -0.52183456, -0.80876911],
       [ 0.73174003,  1.07299907,  0.70228759, -0.91976785],
       [ 1.29731404, -1.15899449,  0.26739685,  0.32497978],
       [ 0.1566397 , -0.1992736 ,  1.01038201, -0.61030675],
       [-1.68379299,  0.44626644,  1.20052603, -0.18622026],
       [ 0.51520292, -0.21870151,  1.22108085,  1.09305973],
       [ 0.20354133, -1.05187875,  0.9538212 , -0.28364463],
       [-0.55506871,  1.62887379,  0.42719155, -0.07394396]])

In [17]:
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.044239,-0.068667,0.595538,-0.138175
std,0.875843,1.052752,0.535565,0.588702
min,-1.683793,-1.251975,-0.521835,-0.919768
25%,-0.377142,-1.027613,0.323598,-0.528641
50%,0.274272,-0.208988,0.564740,-0.130082
75%,0.487670,0.862186,0.996242,0.081859
max,1.297314,1.628874,1.221081,1.093060


In [18]:
df.T

,2020-11-09 14:45:00,2020-11-09 15:45:00,2020-11-09 16:45:00,2020-11-09 17:45:00,2020-11-09 18:45:00,2020-11-09 19:45:00,2020-11-09 20:45:00,2020-11-09 21:45:00,2020-11-09 22:45:00,2020-11-09 23:45:00
A,-0.973261,0.405071,0.345003,0.731740,1.297314,0.156640,-1.683793,0.515203,0.203541,-0.555069
B,1.000826,-0.954815,-1.251975,1.072999,-1.158994,-0.199274,0.446266,-0.218702,-1.051879,1.628874
C,0.394591,0.299934,-0.521835,0.702288,0.267397,1.010382,1.200526,1.221081,0.953821,0.427192
D,0.122288,-0.039428,-0.808769,-0.919768,0.324980,-0.610307,-0.186220,1.093060,-0.283645,-0.073944


In [19]:
df.sort_index(axis=1,ascending=False)

,D,C,B,A
2020-11-09 14:45:00,0.122288,0.394591,1.000826,-0.973261
2020-11-09 15:45:00,-0.039428,0.299934,-0.954815,0.405071
2020-11-09 16:45:00,-0.808769,-0.521835,-1.251975,0.345003
2020-11-09 17:45:00,-0.919768,0.702288,1.072999,0.731740
2020-11-09 18:45:00,0.324980,0.267397,-1.158994,1.297314
2020-11-09 19:45:00,-0.610307,1.010382,-0.199274,0.156640
2020-11-09 20:45:00,-0.186220,1.200526,0.446266,-1.683793
2020-11-09 21:45:00,1.093060,1.221081,-0.218702,0.515203
2020-11-09 22:45:00,-0.283645,0.953821,-1.051879,0.203541
2020-11-09 23:45:00,-0.073944,0.427192,1.628874,-0.555069


In [20]:
df.sort_values(by="C")

,A,B,C,D
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,-0.808769
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,0.324980
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,-0.039428
2020-11-09 14:45:00,-0.973261,1.000826,0.394591,0.122288
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,-0.073944
2020-11-09 17:45:00,0.731740,1.072999,0.702288,-0.919768
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,-0.283645
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,-0.610307
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,-0.186220
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,1.093060


## Selection

### Getting slices

The following show how to get part of the DataFrame (i.e. not just the elements)

In [21]:
## standard and safe
print (df['A'],'\n')

## equivalent but dangerous (imagine blank spaces in the name of the column..)
print (df.A)

2020-11-09 14:45:00   -0.973261
2020-11-09 15:45:00    0.405071
2020-11-09 16:45:00    0.345003
2020-11-09 17:45:00    0.731740
2020-11-09 18:45:00    1.297314
2020-11-09 19:45:00    0.156640
2020-11-09 20:45:00   -1.683793
2020-11-09 21:45:00    0.515203
2020-11-09 22:45:00    0.203541
2020-11-09 23:45:00   -0.555069
Freq: H, Name: A, dtype: float64 

2020-11-09 14:45:00   -0.973261
2020-11-09 15:45:00    0.405071
2020-11-09 16:45:00    0.345003
2020-11-09 17:45:00    0.731740
2020-11-09 18:45:00    1.297314
2020-11-09 19:45:00    0.156640
2020-11-09 20:45:00   -1.683793
2020-11-09 21:45:00    0.515203
2020-11-09 22:45:00    0.203541
2020-11-09 23:45:00   -0.555069
Freq: H, Name: A, dtype: float64


In [22]:
# selecting rows by counting
print (df[0:3])

# or by index
print (df["2020-11-09 14:45:00":"2020-11-09 16:45:00"])

                            A         B         C         D
2020-11-09 14:45:00 -0.973261  1.000826  0.394591  0.122288
2020-11-09 15:45:00  0.405071 -0.954815  0.299934 -0.039428
2020-11-09 16:45:00  0.345003 -1.251975 -0.521835 -0.808769
                            A         B         C         D
2020-11-09 14:45:00 -0.973261  1.000826  0.394591  0.122288
2020-11-09 15:45:00  0.405071 -0.954815  0.299934 -0.039428
2020-11-09 16:45:00  0.345003 -1.251975 -0.521835 -0.808769


### Selection by label

In [23]:
# getting a cross section (part of the DataFrame) using a label
df.loc[dates[0]]

A   -0.973261
B    1.000826
C    0.394591
D    0.122288
Name: 2020-11-09 14:45:00, dtype: float64

In [24]:
# selecting on a multi-axis by label:
df.loc[:,['A','B']]

,A,B
2020-11-09 14:45:00,-0.973261,1.000826
2020-11-09 15:45:00,0.405071,-0.954815
2020-11-09 16:45:00,0.345003,-1.251975
2020-11-09 17:45:00,0.731740,1.072999
2020-11-09 18:45:00,1.297314,-1.158994
2020-11-09 19:45:00,0.156640,-0.199274
2020-11-09 20:45:00,-1.683793,0.446266
2020-11-09 21:45:00,0.515203,-0.218702
2020-11-09 22:45:00,0.203541,-1.051879
2020-11-09 23:45:00,-0.555069,1.628874


In [25]:
# showing label slicing, both endpoints are included:
df.loc['2020-11-09 18:45:00':'2020-11-09 20:45:00',['A','B']]

,A,B
2020-11-09 18:45:00,1.297314,-1.158994
2020-11-09 19:45:00,0.156640,-0.199274
2020-11-09 20:45:00,-1.683793,0.446266


In [26]:
# getting an individual element
print (df.loc[dates[1],'A'])

# equivalently
print (df.at[dates[1],'A'])

0.40507125852194753
0.40507125852194753


### Selecting by position

In [27]:
# select via the position of the passed integers:
print (df.iloc[3],'\n')

# notation similar to numpy/python
print (df.iloc[3:5,0:2])

A    0.731740
B    1.072999
C    0.702288
D   -0.919768
Name: 2020-11-09 17:45:00, dtype: float64 

                            A         B
2020-11-09 17:45:00  0.731740  1.072999
2020-11-09 18:45:00  1.297314 -1.158994


In [28]:
# selecting raws 1,2 and 4 for columns 0 and 2
df.iloc[[1,2,4],[0,2]]

,A,C
2020-11-09 15:45:00,0.405071,0.299934
2020-11-09 16:45:00,0.345003,-0.521835
2020-11-09 18:45:00,1.297314,0.267397


In [29]:
# slicing rows explicitly
print (df.iloc[1:3,:],'\n')

# slicing columns explicitly
print (df.iloc[:,1:3])


                            A         B         C         D
2020-11-09 15:45:00  0.405071 -0.954815  0.299934 -0.039428
2020-11-09 16:45:00  0.345003 -1.251975 -0.521835 -0.808769 

                            B         C
2020-11-09 14:45:00  1.000826  0.394591
2020-11-09 15:45:00 -0.954815  0.299934
2020-11-09 16:45:00 -1.251975 -0.521835
2020-11-09 17:45:00  1.072999  0.702288
2020-11-09 18:45:00 -1.158994  0.267397
2020-11-09 19:45:00 -0.199274  1.010382
2020-11-09 20:45:00  0.446266  1.200526
2020-11-09 21:45:00 -0.218702  1.221081
2020-11-09 22:45:00 -1.051879  0.953821
2020-11-09 23:45:00  1.628874  0.427192


In [30]:
# selecting an individual element by position
df.iloc[1,1]
df.iat[1,1]


-0.9548151584298672

### Boolean index

Very powerful way of filtering out data with certain features. Notation is very similar to numpy arrays.

In [31]:
# Filter by a boolean condition on the values of a single column
df[df['B'] > 0]

,A,B,C,D
2020-11-09 14:45:00,-0.973261,1.000826,0.394591,0.122288
2020-11-09 17:45:00,0.731740,1.072999,0.702288,-0.919768
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,-0.186220
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,-0.073944


In [32]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
df[df>0]

# a DataFrame with the same shape is returned, with NaN's where condition is not met

,A,B,C,D
2020-11-09 14:45:00,NaN,1.000826,0.394591,0.122288
2020-11-09 15:45:00,0.405071,NaN,0.299934,NaN
2020-11-09 16:45:00,0.345003,NaN,NaN,NaN
2020-11-09 17:45:00,0.731740,1.072999,0.702288,NaN
2020-11-09 18:45:00,1.297314,NaN,0.267397,0.324980
2020-11-09 19:45:00,0.156640,NaN,1.010382,NaN
2020-11-09 20:45:00,NaN,0.446266,1.200526,NaN
2020-11-09 21:45:00,0.515203,NaN,1.221081,1.093060
2020-11-09 22:45:00,0.203541,NaN,0.953821,NaN
2020-11-09 23:45:00,NaN,1.628874,0.427192,NaN


### Setting

Combination of selection and setting of values

In [33]:
# setting values by label (same as by position)
df.at[dates[0],'A'] = 0

# setting and assigning a numpy array
df.loc[:,'D'] = np.array([5] * len(df))

# defining a brend new column
df['E'] = np.arange(len(df))*0.5

# defining a brend new column by means of a pd.Series: indexes must be the same!
df['E prime'] = pd.Series(np.arange(len(df))*2, index=df.index)


In [34]:
def dcos(theta):
    theta = theta*(np.pi/180)
    return np.cos(theta)
 
df['cosine'] = pd.Series(df["E"].apply(dcos), index=df.index)
df

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,0,1.000000
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,5,0.5,2,0.999962
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,5,1.0,4,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,6,0.999657
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,5,2.0,8,0.999391
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,5,3.0,12,0.998630
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,5,3.5,14,0.998135
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,5,4.0,16,0.997564
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,5,4.5,18,0.996917


In [35]:
# another example of global setting
df2=df.copy()
df2[df2>0] = -df2
df2

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,-1.000826,-0.394591,-5,0.0,0,-1.000000
2020-11-09 15:45:00,-0.405071,-0.954815,-0.299934,-5,-0.5,-2,-0.999962
2020-11-09 16:45:00,-0.345003,-1.251975,-0.521835,-5,-1.0,-4,-0.999848
2020-11-09 17:45:00,-0.731740,-1.072999,-0.702288,-5,-1.5,-6,-0.999657
2020-11-09 18:45:00,-1.297314,-1.158994,-0.267397,-5,-2.0,-8,-0.999391
2020-11-09 19:45:00,-0.156640,-0.199274,-1.010382,-5,-2.5,-10,-0.999048
2020-11-09 20:45:00,-1.683793,-0.446266,-1.200526,-5,-3.0,-12,-0.998630
2020-11-09 21:45:00,-0.515203,-0.218702,-1.221081,-5,-3.5,-14,-0.998135
2020-11-09 22:45:00,-0.203541,-1.051879,-0.953821,-5,-4.0,-16,-0.997564
2020-11-09 23:45:00,-0.555069,-1.628874,-0.427192,-5,-4.5,-18,-0.996917


### Dropping

N.B.: dropping doesn't act permanently on the DataFrame, i.e. to get that do :
```python
df = df.drop(....)
```

In [36]:
# Dropping by column
df.drop(['E prime'], axis=1)

#which is equivalent to
df.drop(columns=['E prime'])

,A,B,C,D,E,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,1.000000
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,5,0.5,0.999962
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,5,1.0,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,0.999657
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,5,2.0,0.999391
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,5,2.5,0.999048
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,5,3.0,0.998630
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,5,3.5,0.998135
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,5,4.0,0.997564
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,5,4.5,0.996917


In [37]:
# Dropping by raws
# safe and always working
df.drop(df.index[[1,2,3,4]])

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,0,1.000000
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,5,3.0,12,0.998630
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,5,3.5,14,0.998135
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,5,4.0,16,0.997564
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,5,4.5,18,0.996917


In [38]:
# something like df.drop('index_name') 
# would work but the type of index must be specificed, 
# in particular with DatetimeIndex
df.drop(pd.to_datetime("2020-11-09 22:45:00"))

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,0,1.000000
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,5,0.5,2,0.999962
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,5,1.0,4,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,6,0.999657
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,5,2.0,8,0.999391
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,5,3.0,12,0.998630
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,5,3.5,14,0.998135
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,5,4.5,18,0.996917


## Missing data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations.

In [39]:
df_wNan = df[df>0]
df_wNan

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,NaN,1.000826,0.394591,5,NaN,NaN,1.000000
2020-11-09 15:45:00,0.405071,NaN,0.299934,5,0.5,2.0,0.999962
2020-11-09 16:45:00,0.345003,NaN,NaN,5,1.0,4.0,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,6.0,0.999657
2020-11-09 18:45:00,1.297314,NaN,0.267397,5,2.0,8.0,0.999391
2020-11-09 19:45:00,0.156640,NaN,1.010382,5,2.5,10.0,0.999048
2020-11-09 20:45:00,NaN,0.446266,1.200526,5,3.0,12.0,0.998630
2020-11-09 21:45:00,0.515203,NaN,1.221081,5,3.5,14.0,0.998135
2020-11-09 22:45:00,0.203541,NaN,0.953821,5,4.0,16.0,0.997564
2020-11-09 23:45:00,NaN,1.628874,0.427192,5,4.5,18.0,0.996917


In [40]:
# dropping raws with at least a Nan
df_wNan.dropna(how='any')

,A,B,C,D,E,E prime,cosine
2020-11-09 17:45:00,0.73174,1.072999,0.702288,5,1.5,6.0,0.999657


In [41]:
# getting a mask
df_wNan.isna()
#df_wNan.notna()

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,True,False,False,False,True,True,False
2020-11-09 15:45:00,False,True,False,False,False,False,False
2020-11-09 16:45:00,False,True,True,False,False,False,False
2020-11-09 17:45:00,False,False,False,False,False,False,False
2020-11-09 18:45:00,False,True,False,False,False,False,False
2020-11-09 19:45:00,False,True,False,False,False,False,False
2020-11-09 20:45:00,True,False,False,False,False,False,False
2020-11-09 21:45:00,False,True,False,False,False,False,False
2020-11-09 22:45:00,False,True,False,False,False,False,False
2020-11-09 23:45:00,True,False,False,False,False,False,False


In [42]:
# filling missing data
df_wNan.fillna(value=0)

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,0.0,1.000000
2020-11-09 15:45:00,0.405071,0.000000,0.299934,5,0.5,2.0,0.999962
2020-11-09 16:45:00,0.345003,0.000000,0.000000,5,1.0,4.0,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,6.0,0.999657
2020-11-09 18:45:00,1.297314,0.000000,0.267397,5,2.0,8.0,0.999391
2020-11-09 19:45:00,0.156640,0.000000,1.010382,5,2.5,10.0,0.999048
2020-11-09 20:45:00,0.000000,0.446266,1.200526,5,3.0,12.0,0.998630
2020-11-09 21:45:00,0.515203,0.000000,1.221081,5,3.5,14.0,0.998135
2020-11-09 22:45:00,0.203541,0.000000,0.953821,5,4.0,16.0,0.997564
2020-11-09 23:45:00,0.000000,1.628874,0.427192,5,4.5,18.0,0.996917


Fill gaps forward or backward by propagating non-NA values forward or backward:

In [43]:
df_wNan.fillna(method='pad')

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,NaN,1.000826,0.394591,5,NaN,NaN,1.000000
2020-11-09 15:45:00,0.405071,1.000826,0.299934,5,0.5,2.0,0.999962
2020-11-09 16:45:00,0.345003,1.000826,0.299934,5,1.0,4.0,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,6.0,0.999657
2020-11-09 18:45:00,1.297314,1.072999,0.267397,5,2.0,8.0,0.999391
2020-11-09 19:45:00,0.156640,1.072999,1.010382,5,2.5,10.0,0.999048
2020-11-09 20:45:00,0.156640,0.446266,1.200526,5,3.0,12.0,0.998630
2020-11-09 21:45:00,0.515203,0.446266,1.221081,5,3.5,14.0,0.998135
2020-11-09 22:45:00,0.203541,0.446266,0.953821,5,4.0,16.0,0.997564
2020-11-09 23:45:00,0.203541,1.628874,0.427192,5,4.5,18.0,0.996917


## Operations

Here comes the most relevant advantage of DataFrame. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a raw

In [44]:
# Some statistics (mean() just as an example)
# raws
print (df.mean(axis=0),'\n')
# columns
print (df.mean(axis=1),'\n')

A          0.141565
B         -0.068667
C          0.595538
D          5.000000
E          2.250000
E prime    9.000000
cosine     0.998915
dtype: float64 

2020-11-09 14:45:00    1.056488
2020-11-09 15:45:00    1.178593
2020-11-09 16:45:00    1.367292
2020-11-09 17:45:00    2.286669
2020-11-09 18:45:00    2.343587
2020-11-09 19:45:00    2.780971
2020-11-09 20:45:00    2.994518
2020-11-09 21:45:00    3.573674
2020-11-09 22:45:00    3.729007
2020-11-09 23:45:00    4.285416
Freq: H, dtype: float64 



In [45]:
# global operations on columns
df.apply(np.cumsum)

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,0,1.000000
2020-11-09 15:45:00,0.405071,0.046011,0.694525,10,0.5,2,1.999962
2020-11-09 16:45:00,0.750075,-1.205964,0.172690,15,1.5,6,2.999810
2020-11-09 17:45:00,1.481815,-0.132965,0.874978,20,3.0,12,3.999467
2020-11-09 18:45:00,2.779129,-1.291959,1.142375,25,5.0,20,4.998858
2020-11-09 19:45:00,2.935768,-1.491233,2.152757,30,7.5,30,5.997906
2020-11-09 20:45:00,1.251975,-1.044966,3.353283,35,10.5,42,6.996536
2020-11-09 21:45:00,1.767178,-1.263668,4.574363,40,14.0,56,7.994670
2020-11-09 22:45:00,1.970720,-2.315547,5.528185,45,18.0,72,8.992234
2020-11-09 23:45:00,1.415651,-0.686673,5.955376,50,22.5,90,9.989152


In [46]:
df

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,1.000826,0.394591,5,0.0,0,1.000000
2020-11-09 15:45:00,0.405071,-0.954815,0.299934,5,0.5,2,0.999962
2020-11-09 16:45:00,0.345003,-1.251975,-0.521835,5,1.0,4,0.999848
2020-11-09 17:45:00,0.731740,1.072999,0.702288,5,1.5,6,0.999657
2020-11-09 18:45:00,1.297314,-1.158994,0.267397,5,2.0,8,0.999391
2020-11-09 19:45:00,0.156640,-0.199274,1.010382,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.683793,0.446266,1.200526,5,3.0,12,0.998630
2020-11-09 21:45:00,0.515203,-0.218702,1.221081,5,3.5,14,0.998135
2020-11-09 22:45:00,0.203541,-1.051879,0.953821,5,4.0,16,0.997564
2020-11-09 23:45:00,-0.555069,1.628874,0.427192,5,4.5,18,0.996917


In [47]:
df.apply(lambda x: x.max() - x.min())

A           2.981107
B           2.880849
C           1.742915
D           0.000000
E           4.500000
E prime    18.000000
cosine      0.003083
dtype: float64

In [48]:
# syntax is as usual similar to that of numpy arrays
df['A']+df['B']

2020-11-09 14:45:00    1.000826
2020-11-09 15:45:00   -0.549744
2020-11-09 16:45:00   -0.906972
2020-11-09 17:45:00    1.804739
2020-11-09 18:45:00    0.138320
2020-11-09 19:45:00   -0.042634
2020-11-09 20:45:00   -1.237527
2020-11-09 21:45:00    0.296501
2020-11-09 22:45:00   -0.848337
2020-11-09 23:45:00    1.073805
Freq: H, dtype: float64

Let's play it hard and load (in memory) a (relatively) large dataset

In [49]:
# WARNING! link in past notebook was wrong!, (if needed) get the right file from:
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

file_name="~/data/data_000637.txt"
data=pd.read_csv(file_name)
data

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdb in position 0: invalid continuation byte

Let's now do some operations among (elements of) columns

In [ ]:
# the one-liner killing it all
data['timens']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25

In [ ]:
# the old slooow way
def conversion(data):
    result=[]
    for i in range(len(data)): 
        result.append(data.loc[data.index[i],'TDC_MEAS']*25/30.+data.loc[data.index[i],'BX_COUNTER']*25)
    return result

data['timens']=conversion(data)

## Merge

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

### Concat

concatenation (adding rows) is straightforward


In [ ]:
rdf = pd.DataFrame(np.random.randn(10, 4))
rdf

In [ ]:
# divide it into pieaces raw-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
pieces

In [ ]:
# put it back together
#pd.concat(pieces)

# indexes can be ignored
pd.concat(pieces, ignore_index=True)

# in case of dimension mismatch, Nan are added where needed

In [ ]:
# appending a single row (as a Series)
s = rdf.iloc[3]
rdf.append(s, ignore_index=True)
rdf

### Merge/Join

SQL like operations on table can be performed on DataFrames. This is all rather sophisticated, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left,right,on="key")

## Grouping

By “group by” we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [3]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})
gdf

,A,B,C,D
0,foo,one,-1.492475,-0.117190
1,bar,one,-1.247712,-0.459805
2,foo,two,2.014393,0.975304
3,bar,three,-0.034368,0.077099
4,foo,two,-0.796964,0.996129
5,bar,two,-2.349403,0.269032
6,foo,one,-0.523326,1.478050
7,foo,three,0.774616,-0.172173


In [4]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where number are there).
gdf.groupby('A').sum()

,C,D
A,,
bar,-3.631483,-0.113673
foo,-0.023755,3.160120


## Multi-indexing


Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In essence, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1d) and DataFrame (2d).

In [ ]:
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print (multi_index,'\n')

s = pd.Series(np.random.randn(8), index=multi_index)
print (s)


In [5]:
# it enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A','B']).sum()

C         D
A   B                        
bar one   -1.247712 -0.459805
    three -0.034368  0.077099
    two   -2.349403  0.269032
foo one   -2.015801  1.360860
    three  0.774616 -0.172173
    two    1.217429  1.971433

In [6]:
# stack() method “compresses” a level in the DataFrame’s columns
gdf.groupby(['A','B']).sum().stack()

A    B       
bar  one    C   -1.247712
            D   -0.459805
     three  C   -0.034368
            D    0.077099
     two    C   -2.349403
            D    0.269032
foo  one    C   -2.015801
            D    1.360860
     three  C    0.774616
            D   -0.172173
     two    C    1.217429
            D    1.971433
dtype: float64

## Plotting

Just a preview, more on the next lab class!

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts.cumsum().plot()

In [ ]:
import matplotlib.pyplot as plt

pdf=pd.DataFrame(np.random.randn(1000, 4), index=ts.index,columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best')